In [1]:
import pandas as pd
import numpy as np
from dateutil.parser import parse

#import and read data
samples = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")
analyses = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Horizontes Analises.xlsx")
morphology = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Horizontes_Morfologia.xlsx")
profile_loc = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")
soil_profile = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")
elemental_analyses = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Data XRF Angola_inicial.xlsx")
#soil_type = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

# Samples table

In [38]:
samples = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/AmostrasAngolaTerrario.xlsx")

# Rename columns in the DataFrame
samples.rename(columns={
    'Registo': 'sample_id',
    'Nº Campo': 'site_info_id',
    'Ano': 'year',
    'Perfil': 'profile_id',
    'Campanha': 'campaign',
    'Colónia_Pais': 'country',
    'Distrito': 'district',
    'AmostraCrivada': 'sample_sifted',
    'AmostraNaoCrivada': 'sample_not_sifted',
    'Prateleira': 'shelf',
    'Sala': 'room'
}, inplace=True)

# Replace nulls with the string 'null'
samples_filled = samples.fillna('null')

# analyse duplicated sample_id (PK)
duplicated_values = samples['sample_id'][samples['sample_id'].duplicated()].unique()
print(duplicated_values)

# Change PK (sample_id) to a consistent datatype: Converted to string and truncate to 20 characters (similar to VARCHAR(20))
samples_filled['sample_id'] = samples_filled['sample_id'].astype(str).str.strip().str[:20]

## make sure profile column is a string/varchar
samples_filled['profile_id'] = samples_filled['profile_id'].astype(str).str.strip().str[:20]

# Drop unnecessary columns
samples_clean = samples_filled.drop(columns=[
    'campaign', 'country', 'Província', 'sample_not_sifted', 'sample_sifted', 'Obs'
])

samples_clean = samples_filled

## add empty FK columns to populate later
samples_clean['lab_info_id'] = pd.NA
samples_clean['horizon_id'] = pd.NA

# Rename for consistent schema (only if needed)
samples_clean = samples_clean.rename(columns={
    'profile': 'profile',
    'shelf': 'shelf',
    'room': 'Room',
    'year': 'year'
})

samples_clean['shelf'] = samples_clean['shelf'].astype("string")


# Reorder columns to match SAMPLES schema
samples_check = samples_clean[[
    'sample_id',
    'site_info_id',
    'profile_id',
    'horizon_id',
    'shelf',
    'Room',
    'year'
]]

## Preview results:
samples_check.head()

[]


,sample_id,site_info_id,profile_id,horizon_id,shelf,Room,year
0,630,172,139,<NA>,1,22,1946.0
1,631,173,139,<NA>,1,22,1946.0
2,632,174,139,<NA>,1,22,1946.0
3,633,175,139,<NA>,1,22,1946.0
4,687,1034,208,<NA>,1,22,1946.0


# Analyses table

In [39]:
analyses.rename(columns={
    'Amostra': 'sample_id',
    'Morfo_id': 'horizon_id',
    'Analise-id': 'analysis_id',
    'PERFIL': 'profile_id',
    'LS': 'upper_limit',
    'LI': 'lower_limit',
    'EG': 'EG',
    'AG': 'thick_clay',
    'AF': 'fine_clay',
    'Argila': 'clay',
    'L': 'silt',
    'Gesso': 'gypsum',
    'Fe livre': 'free_iron',
    'CO':'organic_carbon',
    'N total': 'total_N',
    'MO': 'OM',
    'Soma de bases': 'exchangable_bases_sum',
    'CTC': 'CEC',
    'Cloretos':'chlorides',
    'Sulfatos':'sulfates',
    'Condutividade':'conductivity',
    'Sódio solúvel':'soluble_sodium',
    'P205 total': 'P205',
    'pH (H2O)':'pH_H2O',
    'pH(KCL)':'pH_KCL',
    'MO': 'organic_material'
}, inplace=True)

# Replace nulls with the string 'null'
analyses_filled = analyses.fillna('null')

## make sure profile column is a string/varchar
analyses_filled['profile_id'] = analyses_filled['profile_id'].astype(str).str.strip().str[:20]

# Filter out and print duplicated samples (including the first occurrence)
duplicated_values = analyses['sample_id'][analyses['sample_id'].duplicated()].unique()
print(duplicated_values)

# Drop columns not needed
analyses_drop = analyses.drop([
    'EqMol (SiO2)', 'EqMol(Al2O3)', 'EqMol(Fe2O3)', 'SiO2/Al2O3', 
    'SiO2/Fe2O3', 'SiO2/R2O3', 'Fe2O3/Al2O3', 'FE2O3_TARG', 
    'FE2O3_LARG', 'CEC_ARG', 'GR', 'ID', 'COD_PROV'
], axis=1)

# Add a new Primary Key ID column starting from 1
analyses_drop.insert(0, 'lab_sample_id', range(1, len(analyses_drop) + 1))

# Final cleaned dataframe
analyses2 = analyses_drop

# Define the desired final column order
final_columns = [
    'lab_sample_id',
    'sample_id',
    'minerology_id',
    'EG',
    'thick_clay',
    'fine_clay',
    'silt',
    'clay',
    'Eq_Hum',
    'atm_1/3',
    'atm_15',
    'CACO3',
    'gypsum',
    'free_iron',
    'organic_carbon',
    'total_N',
    'P205',
    'organic_material',
    'pH_H2O',
    'pH_KCL',
    'Ca++',
    'Mg++',
    'Na+',
    'K+',
    'exchangable_bases_sum',
    'CEC',
    'V',
    'conductivity',
    'soluble_sodium',
    'Min_<0,002',
    'Min_0,05-0,02',
    'Min_0,2-0,05',
    'Min_2-0,2',
]


# Add missing columns with NaN values
for col in final_columns:
    if col not in analyses2.columns:
        analyses2[col] = pd.NA  # or np.nan if you prefer

# Reorder columns
analyses_ready = analyses2[final_columns]

# Preview first 10 rows
analyses_ready.head(10)


[10337.    nan 11608.  3497.  3498.  2923.  2924.  2925.  2926.  2927.
  2928.  2929.  2930. 16355. 16356.   419.   420. 15682. 13817. 13965.
  3418.  9230.  8639.  3529.]


,lab_sample_id,sample_id,minerology_id,EG,thick_clay,fine_clay,silt,clay,Eq_Hum,atm_1/3,atm_15,CACO3,gypsum,free_iron,organic_carbon,total_N,P205,organic_material,pH_H2O,pH_KCL,Ca++,Mg++,Na+,K+,exchangable_bases_sum,CEC,V,conductivity,soluble_sodium,"Min_<0,002","Min_0,05-0,02","Min_0,2-0,05","Min_2-0,2"
0,1,10999.0,<NA>,NaN,61.700001,32.8,0.2,5.3,4.600000,NaN,NaN,NaN,NaN,NaN,0.55,NaN,0.04,0.9,5.4,4.2,0.23,0.14,0.02,0.03,NaN,1.83,23.000000,NaN,0.0,NaN,NaN,NaN,NaN
1,2,11000.0,<NA>,NaN,52.799999,35.1,0.7,11.4,6.400000,NaN,NaN,NaN,NaN,NaN,0.33,NaN,0.05,0.6,5.1,4.2,0.04,0.10,0.04,0.03,NaN,1.98,10.600000,NaN,0.0,NaN,NaN,NaN,NaN
2,3,11001.0,<NA>,NaN,42.500000,46.2,0.2,11.1,6.300000,NaN,NaN,NaN,NaN,NaN,0.21,NaN,0.05,0.4,5.1,4.2,0.02,0.03,0.03,0.01,NaN,1.62,5.600000,NaN,0.0,NaN,NaN,NaN,NaN
3,4,11002.0,<NA>,NaN,42.599998,41.8,0.2,15.4,5.200000,NaN,NaN,NaN,NaN,NaN,0.10,NaN,0.05,0.2,5.1,4.2,0.02,0.02,0.03,0.01,NaN,0.91,8.800000,NaN,0.0,NaN,NaN,NaN,NaN
4,5,11003.0,<NA>,NaN,36.799999,47.5,1.2,14.5,7.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.2,0.03,0.03,0.03,0.01,NaN,1.04,7.700000,NaN,0.0,NaN,NaN,NaN,NaN
5,6,11004.0,<NA>,NaN,45.099998,41.8,1.4,11.7,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.9,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
6,7,11011.0,<NA>,1.0,15.300000,48.5,17.6,18.5,19.700001,NaN,3.0,NaN,NaN,1.45,1.11,NaN,0.05,1.9,5.9,4.5,0.51,0.75,0.02,0.18,NaN,4.49,32.500000,NaN,0.0,NaN,NaN,NaN,NaN
7,8,11012.0,<NA>,3.0,18.600000,42.3,17.8,21.3,19.600000,NaN,7.1,NaN,NaN,1.56,0.57,NaN,0.04,1.0,5.5,4.2,0.07,0.42,0.05,0.07,NaN,2.96,20.600000,NaN,0.0,NaN,NaN,NaN,NaN
8,9,11013.0,<NA>,7.0,12.000000,36.8,22.1,29.1,21.700001,NaN,10.0,NaN,NaN,2.34,0.26,NaN,0.04,0.4,5.3,4.2,0.07,0.38,0.02,0.04,NaN,3.02,16.900000,NaN,0.0,NaN,NaN,NaN,NaN
9,10,11014.0,<NA>,21.0,11.000000,34.8,21.6,32.6,21.600000,NaN,11.1,NaN,NaN,2.68,0.15,NaN,0.04,0.3,5.4,4.3,0.07,0.45,0.05,0.03,NaN,3.11,19.299999,NaN,0.0,NaN,NaN,NaN,NaN


## joining elemental analyses to analyses


In [4]:
# Rename 'Lab. Code' to 'sample_id'
elemental_analyses.rename(columns={'Lab. Code': 'sample_id'}, inplace=True)

# Temporarily convert to string to do the replacements
sample_id_cleaned = elemental_analyses['sample_id'].astype(str)
sample_id_cleaned = sample_id_cleaned.str.replace('C-', '', regex=False)
sample_id_cleaned = sample_id_cleaned.str.replace(' MNL', '', regex=False)

# Then convert to numeric (int or float, depending on your data)
elemental_analyses['sample_id'] = pd.to_numeric(sample_id_cleaned, errors='coerce')

#rename to field_sample_code
elemental_analyses.rename(columns={
    'Code': 'field_sample_code'
}, inplace=True)

In [5]:
print (analyses_ready['sample_id'].dtype)
print (elemental_analyses['sample_id'].dtype)


float64
int64


In [6]:
elemental_analyses['sample_id'] = elemental_analyses['sample_id'].astype(float)


In [7]:
# Check which sample_ids in elemental_analyses are NOT in analyses_ready
missing_ids = elemental_analyses[~elemental_analyses['sample_id'].isin(analyses_ready['sample_id'])]

# Display them
print("Sample IDs in elemental_analyses not found in analyses_ready:")
print(missing_ids['sample_id'].unique())


Sample IDs in elemental_analyses not found in analyses_ready:
[16105. 16217. 16033. 16231. 16208. 16274. 16225. 15731. 16078. 15786.
 15693. 15678. 15959. 16372. 16418. 16459. 16477. 15498. 15437. 12749.
 13109. 14337. 14319. 15484. 15508. 15614. 15580. 15582. 15606. 15463.
 14496. 11343. 17892. 16976. 17269. 18248. 16946. 17686. 17728. 17415.
 17642. 18212. 17016. 17282. 18429. 17340. 18824. 18448.  8721.  7313.
  5502.  5392.  7317.  8626.  7193.  7254.  7293.  8240. 15397. 14403.
 14983. 16126. 15059. 15322. 15268. 14808. 15106.]


LEFT JOIN ELEMENTAL ANALYSE TO ANALYSE TABLE

In [8]:
merged1= analyses_ready.merge(elemental_analyses, on='sample_id', how='left')
merged1.head()


,lab_sample_id,sample_id,minerology_id,EG,thick_clay,fine_clay,silt,clay,Eq_Hum,atm_1/3,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,10999.0,<NA>,NaN,61.700001,32.8,0.2,5.3,4.6,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,11000.0,<NA>,NaN,52.799999,35.1,0.7,11.4,6.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,11001.0,<NA>,NaN,42.500000,46.2,0.2,11.1,6.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,11002.0,<NA>,NaN,42.599998,41.8,0.2,15.4,5.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,11003.0,<NA>,NaN,36.799999,47.5,1.2,14.5,7.3,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
print(analyses_ready[analyses_ready['sample_id'] == 6940])
print(elemental_analyses[elemental_analyses['sample_id'] == 6940])


      lab_sample_id  sample_id minerology_id  EG  thick_clay  fine_clay  silt  \
1361           1362     6940.0          <NA> NaN        31.1       43.8   2.7   

      clay  Eq_Hum  atm_1/3  ...  K+  exchangable_bases_sum  CEC   V  \
1361  22.7    18.9      NaN  ... NaN                    NaN  NaN NaN   

      conductivity  soluble_sodium  Min_<0,002  Min_0,05-0,02  Min_0,2-0,05  \
1361           NaN             0.0         NaN            NaN           NaN   

      Min_2-0,2  
1361        NaN  

[1 rows x 33 columns]
    sample_id Soil Profile field_sample_code Code.1      Depht Sampling Date  \
80     6940.0      P134/59          A-320/59  S-492  1.30-1.70    1954-08-06   

             Mg             Al             Si      P  ...    Ta   W  Pt  Au  \
80  1457.666667  106266.333333  308859.666667  468.0  ...  26.0 NaN NaN NaN   

           Hg   Tl         Pb  Bi         Th   U  
80  10.666667  4.0  25.333333 NaN  12.333333 NaN  

[1 rows x 44 columns]


In [10]:
# Define final column list
final_columns = [
    'lab_sample_id', 'sample_id', 'EG', 'thick_clay', 'fine_clay', 'silt', 'clay', 'Eq_Hum', 'atm_1/3', 'atm_15',
    'CACO3', 'gypsum', 'free_iron', 'organic_carbon', 'total_N', 'P205', 'organic_material', 'pH_H2O', 'pH_KCL',
    'Ca++', 'Mg++', 'Na+', 'K+', 'exchangable_bases_sum', 'CEC', 'V', 'conductivity', 'soluble_sodium', 'Min_<0,002',
    'Min_0,05-0,02', 'Min_0,2-0,05', 'Min_2-0,2', 
    'field_sample_code', 'Depth', 'Al', 'Si', 'P', 'S', 'Cl', 'Ti', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'As', 'Se', 'Rb', 'Sr', 'Zr', 'Nb', 'Mo', 'Cd', 'Sn', 'Sb', 'Ba', 'Ta', 'W', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi',
    'Th', 'U',
]

# Ensure all columns exist in the DataFrame
for col in final_columns:
    if col not in merged1.columns:
        merged1[col] = pd.NA  # or np.nan if preferred

# Reorder DataFrame columns
merged1 = merged1[final_columns]

# Check and print the datatypes of each column
print("Data types of each column in merged1:\n")
print(merged1.dtypes)

Data types of each column in merged1:

lab_sample_id      int64
sample_id        float64
EG               float64
thick_clay       float64
fine_clay        float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 68, dtype: object


### Check/Change datatypes

In [11]:
# Change PK (sample_id) to a consistent datatype: Converted to string and truncate to 20 characters (similar to VARCHAR(20))
merged1['sample_id'] = merged1['sample_id'].astype(str).str.strip().str[:20]
merged1['lab_sample_id'] = merged1['lab_sample_id'].astype(str).str.strip().str[:20]

print(merged1.dtypes)

lab_sample_id     object
sample_id         object
EG               float64
thick_clay       float64
fine_clay        float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 68, dtype: object


### Check and/or add FK

### Save to CSV

In [12]:
merged1.head()

,lab_sample_id,sample_id,EG,thick_clay,fine_clay,silt,clay,Eq_Hum,atm_1/3,atm_15,...,Ta,W,Pt,Au,Hg,Tl,Pb,Bi,Th,U
0,1,10999.0,NaN,61.700001,32.8,0.2,5.3,4.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,11000.0,NaN,52.799999,35.1,0.7,11.4,6.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,11001.0,NaN,42.500000,46.2,0.2,11.1,6.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,11002.0,NaN,42.599998,41.8,0.2,15.4,5.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,11003.0,NaN,36.799999,47.5,1.2,14.5,7.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merged1.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/analyses_clean.csv", index=False)

# Soil Profile Table

In [42]:
## rename columns
soil_profile = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_solo.xlsx")

#rename columns
soil_profile.rename(columns={
    'Perfil': 'profile_id',
    'Agrupamento': 'grouping',
    'Pro': 'province',
    'País': 'country',
    'Local': 'location',
    'DATA': 'date',
    'CEP_NOME': 'CEP_NAME',
}, inplace=True)

# Drop unnecessary columns
soil_profile_cleaning1 = soil_profile.drop(columns=[
    'grouping', 'REF', 'province', 'country', 'LOCAL', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3', 'CEP_GR'
])

soil_profile_cleaning2 = soil_profile.drop(columns=[
    'grouping', 'REF', 'province', 'country', 'LOCAL', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3', 'date', 'CEP_GR'
])

## make sure profile_id column is a string/varchar
soil_profile_cleaning1['profile_id'] = soil_profile_cleaning1['profile_id'].astype(str).str.strip().str[:20]

# Add missing columns
soil_profile_cleaning2['site_info_id'] = pd.NA
soil_profile_cleaning2['sample_id'] = pd.NA 
soil_profile_cleaning2['soil_type_id'] = pd.NA  
soil_profile_cleaning2['site_info_id'] = soil_profile_cleaning2['site_info_id'].astype(str)


# Reorder columns to match SAMPLES schema
profile_clean = soil_profile_cleaning2[[
    'profile_id',
    'site_info_id',
    'sample_id',
    'soil_type_id'
]]

# Preview the result
profile_clean.head()
#profile_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/profile_clean.csv", index=False)


,profile_id,site_info_id,sample_id,soil_type_id
0,1/51,<NA>,<NA>,<NA>
1,1/57,<NA>,<NA>,<NA>
2,1/59,<NA>,<NA>,<NA>
3,1/63,<NA>,<NA>,<NA>
4,10/54,<NA>,<NA>,<NA>


In [43]:
soil_profile_cleaning1 = soil_profile_cleaning1[[
    'profile_id',
    'date'
]]

soil_profile_cleaning1.head()

,profile_id,date
0,1/51,1951-08-14
1,1/57,1957-06-26
2,1/59,1959-07-07
3,1/63,1963-06-25
4,10/54,1954-07-22


# Morphology Horizon Table

In [44]:
#rename columns
morphology.rename(columns={
    'Morfo_id':'horizon_id',
    'Amostra': 'sample_id',
    'Perfil': 'profile_id',
    'CM':'horizon_layer',
    'Limite Superior': 'upper_depth',
    'Limite inferior': 'lower_depth',
    'Grau de humidade': 'moisture_degree',
    'Quantidade de raízes': 'root_quantity',
    'Diâmetro de raízes': 'root_diameter',
    'Textura': 'texture',
    'Tipo de estrutura': 'structure_type',
    'Classes de estrutura': 'structure_class',
    'Grau de estrutura': 'structure_degree',
    'Diâmetro de poros': 'pore_diameter',
    'Quantidade de poros': 'pore_quantity',
    'Forma de poros': 'pore_shape',
    'Cor (s)': 'dry_color_name',
    'Matiz (s)': 'dry_hue',
    'Valor (s)':'dry_value',
    'Croma (s)': 'dry_chroma',
    'Cor (h)': 'moist_color_name',
    'Matiz (h)': 'moist_hue',
    'Valor (h)': 'moist_value',
    'Croma (h)': 'moist_chroma',
    'Compacidade':'compaction',
    'Dureza': 'durability'
}, inplace=True)

# Drop unnecessary columns
morphology_cleaning = morphology.drop(columns=[
    'ID1', 'Agrupamento', 'REF', 'Pro', 'Observaçoes', 'Horizonte de diagnóstico', 'Propriedade de diagnóstico', 'Nitidez do limite', 'Designação do horizonte', 'Observaçoes', 'Confirmar', 'Adesividade', 'Plasticidade', 'Efervescência com HCl', 'Friabilidade', 'Orientação das Fendas', 'Largura das fendas', 'Quantidade de fendas'
])

## make sure profile_id column is a string/varchar
morphology_cleaning['profile_id'] = morphology_cleaning['profile_id'].astype(str).str.strip().str[:20]


#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
morphology_cleaning = morphology_cleaning.applymap(remove_accents)

morphology_cleaning = morphology_cleaning.fillna('null')


# Reorder columns to match SAMPLES schema
morphology_clean = morphology_cleaning[[
    'horizon_id',
    'sample_id',
    'profile_id',
    'horizon_layer',
    'upper_depth',
    'lower_depth',
    'moisture_degree',
    'root_quantity',
    'root_diameter',
    'texture',
    'structure_type',
    'structure_class',
    'structure_degree',
    'pore_diameter',
    'pore_quantity',
    'pore_shape',
    'dry_color_name',
    'dry_hue',
    'dry_value',
    'dry_chroma',
    'moist_color_name',
    'moist_hue',
    'moist_value',
    'moist_chroma',
    'compaction',
    'durability'
]]

# Show first few rows
morphology_clean.head(5)


,horizon_id,sample_id,profile_id,horizon_layer,upper_depth,lower_depth,moisture_degree,root_quantity,root_diameter,texture,structure_type,structure_class,structure_degree,pore_diameter,pore_quantity,pore_shape,dry_color_name,dry_hue,dry_value,dry_chroma,moist_color_name,moist_hue,moist_value,moist_chroma,compaction,durability
0,B_101/62_1_1,10999.0,101/62,1.0,0.0,11.0,Seco,Muitas finas e bastantes medias,null,Arenoso,Granulosa,Fina (1 a 2 mm),Fraco,Muito finos,Pouco poroso,null,Pardo-acinzentado a pardo,10YR,5.0,2.5,Pardo-acinzentado-escuro,10YR,4.0,2.0,Pequena a minima,Brando
1,B_101/62_2_1,11000.0,101/62,2.0,11.0,28.0,Seco,Bastantes finas e medias e raras grossas,null,Arenoso-franco,null,null,Sem agregacao,Muito finos,Pouco poroso,null,Pardo,10YR,5.0,3.0,Pardo-amarelado-escuro,10YR,3.0,4.0,Pequena,Brando
2,B_101/62_3_1,11001.0,101/62,3.0,28.0,54.0,Seco,Algumas finas e medias e raras grossas,null,Arenoso-franco,null,null,Sem agregacao,Muito finos,Pouco poroso,null,Pardo-amarelado-claro,10YR,6.0,4.0,Pardo-amarelado-escuro,10YR,4.0,4.0,Pequena a minima,Brando
3,B_101/62_4_1,11002.0,101/62,4.0,54.0,90.0,Seco,"Poucas finas, algumas medias e raras grossas",null,Franco-arenoso a arenoso-franco,null,null,Sem agregacao,Muito finos,Pouco a medianamente poroso,null,Amarelo a amarelo-avermelhado,"8,75YR",7.0,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena a minima,Brando
4,B_101/62_5_2,11003.2,101/62,5.0,90.0,160.0,Seco a humido,Raras,Medias e grossas,Arenoso-franco,null,null,Sem agregacao,Muito finos,Pouco a medianamente poroso,null,Pardo-avermelhado,"7,5YR",7.0,6.0,Pardo-forte,"7,5YR",5.0,6.0,Pequena,Brando


# Soil Type Table

In [46]:
#rename columns
soil_type = soil_profile

soil_type.rename(columns={
    'Perfil': 'profile_id',
    'Agrupamento': 'grouping',
    'Pro': 'province',
    'País': 'country',
    'Local': 'location',
    'DATA': 'date',
    'CEP_NOME': 'CEP_NAME',
}, inplace=True)

# Drop unnecessary columns
soil_type_cleaning = soil_type.drop(columns=[
    'REF', 'province', 'country', 'LOCAL', 'DESCRITOR1', 'DESCRITOR2', 'DESCRITOR3', 'date', 'Fase', 'D_INSERÇAO', 'Publicação', 'WRB_old', 'Missão'
])
# Add a new Primary Key ID column starting from 1
soil_type_cleaning.insert(0, 'soil_type_id', range(1, len(soil_type_cleaning) + 1))

## make sure profile_id column is a string/varchar
soil_type_cleaning['profile_id'] = soil_type_cleaning['profile_id'].astype(str).str.strip().str[:20]

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
soil_type_cleaning = soil_type_cleaning.applymap(remove_accents)

soil_type_clean = soil_type_cleaning

soil_type_clean.head(2)


,soil_type_id,profile_id,grouping,CEP_GR,CEP_NAME,FAO
0,1,1/51,H 62,NaN,NaN,NaN
1,2,1/57,M 9,Aridicos,Aridicos com calcario Pardo-cinzentos,CLha


# Site info table

In [47]:
site_info_cleaning = profile_loc

site_info_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'X_COORD': 'X_coord',
    'Y_COORD': 'Y_coord'
}, inplace=True)

# Add a new Primary Key ID column starting from 1
site_info_cleaning.insert(0, 'site_info_id', range(1, len(site_info_cleaning) + 1))

## make sure profile_id column is a string/varchar
site_info_cleaning['profile_id'] = site_info_cleaning['profile_id'].astype(str).str.strip().str[:20]


# Add missing columns
site_info_cleaning['land_cover_id'] = pd.NA
site_info_cleaning['climate_id'] = pd.NA  
site_info_cleaning['geology_id'] = pd.NA
site_info_cleaning['topo_feature_id'] = pd.NA  
site_info_cleaning['sampling_date'] = pd.NA
site_info_cleaning['districts_id'] = pd.NA  


site_info_clean = site_info_cleaning[[
    'site_info_id',
    'profile_id', ##do I add profile ID here to then try to merge date info from soil Perfis_solo table? or is this too complicated?
    'X_coord',
    'Y_coord',
    'land_cover_id',
    'climate_id',
    'geology_id',
    'topo_feature_id',
    'sampling_date',
    'districts_id'
]]

In [48]:
# Merging date info from perfis_solo to site_info table

merged = soil_profile_cleaning1.merge(site_info_clean, on='profile_id', how='left')

merged['site_info_id'] = merged['site_info_id'].astype(str)


pd.set_option('display.max_columns', None)

soil_profile_clean = merged
soil_profile_clean.head()


,profile_id,date,site_info_id,X_coord,Y_coord,land_cover_id,climate_id,geology_id,topo_feature_id,sampling_date,districts_id
0,1/51,1951-08-14,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/57,1957-06-26,1.0,12.161278,-15.222598,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,1/59,1959-07-07,2.0,12.575775,-4.866986,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,1/63,1963-06-25,4.0,17.081955,-9.274587,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,10/54,1954-07-22,9.0,14.445188,-14.922688,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


Save clean table

# Climate table


In [49]:
import pandas as pd
import unicodedata

# Load original Excel data
profile_loc = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")

# Create a copy for cleaning
climate_features_cleaning = profile_loc.copy()

# Mapping from climate codes to descriptions
code_to_description = {
    "B1": "Húmido", "B2": "Húmido", "B3": "Húmido", "B4": "Húmido",
    "C1": "Sub-húmido seco", "C2": "Sub-húmido chuvoso",
    "D": "Semi-árido", "E": "Árido", "Aw": "Tropical chuvoso com estação seca no Inverno, de savana",
    "BSw": "Seco de estepe, com chuva predominante no Verão",
    "BWw": "Seco de deserto, com chuva predominante no Verão",
    "Cw": "Mesotérmico húmido com estação seca no Inverno",
    "ARe": "Arídico extremo", "ARf": "Arídico fraco", "ARt": "Arídico típico",
    "tUDs": "Tempúdico seco", "tUSh": "Tempústico húmido", "tUSt": "Tempústico típico",
    "TUDs": "Tropúdico seco", "TUSa": "Tropústico arídico", "TUSu": "Tropústico údico", "TUSt": "Tropustico típico",
    "H": "Hipertérmico", "iH": "Iso-Hipertérmico", "iT": "Iso-Térmico", "T": "Térmico"
}

# Replace climate codes with descriptions
for col in ["CL_THORNTH", "CL_KOPPEN", "REG_HÍDRIC", "REG_TÉRMIC"]:
    climate_features_cleaning[col] = climate_features_cleaning[col].replace(code_to_description)

# Function to average values like "21-22" -> 21.5
def average_range(value):
    if isinstance(value, str) and '-' in value:
        try:
            nums = [float(x.strip()) for x in value.split('-')]
            return sum(nums) / len(nums)
        except:
            return None
    try:
        return float(value)
    except:
        return None

# Apply to TMA and PMA columns
climate_features_cleaning["TMA"] = climate_features_cleaning["TMA"].apply(average_range)
climate_features_cleaning["PMA"] = climate_features_cleaning["PMA"].apply(average_range)

# Rename columns
climate_features_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'CL_THORNTH': 'thornthwaite_climate',
    'CL_KOPPEN': 'koppen_climate',
    'TMA': 'mean_annual_temp',
    'PMA': 'mean_annual_precip',
    'REG_HÍDRIC': 'hydric_regime',
    'REG_TÉRMIC': 'thermal_regime'
}, inplace=True)

## make sure profile_id column is a string/varchar
climate_features_cleaning['profile_id'] = climate_features_cleaning['profile_id'].astype(str).str.strip().str[:20]

# Drop accents from text values
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

climate_features_cleaning = climate_features_cleaning.applymap(remove_accents)

# Add primary key
climate_features_cleaning.insert(0, 'climate_id', range(1, len(climate_features_cleaning) + 1))

# Select final columns
climate_features_clean = climate_features_cleaning[[
    'climate_id',
    'profile_id',
    'mean_annual_temp',
    'mean_annual_precip',
    'koppen_climate',
    'thornthwaite_climate',
    'hydric_regime',
    'thermal_regime'
]]

# Preview
climate_features_clean.head()


,climate_id,profile_id,mean_annual_temp,mean_annual_precip,koppen_climate,thornthwaite_climate,hydric_regime,thermal_regime
0,1,1/57,NaN,NaN,NaN,Arido,NaN,NaN
1,2,1/59,NaN,NaN,NaN,Humido,NaN,NaN
2,3,1/61,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1/63,21.5,1500.0,"Tropical chuvoso com estacao seca no Inverno, ...",Humido,Tropustico udico,Iso-Hipertermico
4,5,1/64,NaN,NaN,NaN,NaN,NaN,NaN


Check FK relationships and datatypes

In [50]:
# Check and print the datatypes of each column
print("Data types of each column in profile_clean:\n")
print(climate_features_clean.dtypes)

Data types of each column in profile_clean:

climate_id                int64
profile_id               object
mean_annual_temp        float64
mean_annual_precip      float64
koppen_climate           object
thornthwaite_climate     object
hydric_regime            object
thermal_regime           object
dtype: object


# Topo features table

In [51]:
# Create a copy for cleaning
topo_features_cleaning = profile_loc.copy()

# Rename columns for clarity
topo_features_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'TOPOGRAFIA': 'slope_code',
    'ALTITUDE': 'altitude',
}, inplace=True)

# Add missing columns
topo_features_cleaning['aspect'] = pd.NA  
topo_features_cleaning['land_surface_temp'] = pd.NA
topo_features_cleaning['dem_elevation'] = pd.NA  

# Add primary key column
topo_features_cleaning.insert(0, 'topo_features_id', range(1, len(topo_features_cleaning) + 1))

## make sure profile_id column is a string/varchar
topo_features_cleaning['profile_id'] = topo_features_cleaning['profile_id'].astype(str).str.strip().str[:20]

# Create slope class mapping dictionary
slope_code_to_description = {
    "D1": "Plano (Declives < 2%)",
    "D2": "Ondulado muito suave (Declives > 2% e < 3%)",
    "D3": "Ondulado suave (Declives > 3% e < 5%)",
    "D4": "Ondulado (Declives > 5% e < 8%)",
    "D5": "Acidentado (Declives > 8% e < 15%)",
    "D6": "Escarpado (Declives >15% e < 30%)",
    "D7": "Montanhoso (Declives > 30%)"
}

# Create a mapping DataFrame for slope classes
slope_classes_df = pd.DataFrame([
    {"slope_code": code, "slope_description": desc}
    for code, desc in slope_code_to_description.items()
])

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
topo_features_cleaning = topo_features_cleaning.applymap(remove_accents)

# Final cleaned topo features table (referencing slope_code, not description)
topo_features_clean = topo_features_cleaning[[
    'topo_features_id',
    'profile_id', 
    'slope_code',
    'altitude',
    'aspect',
    'land_surface_temp',
    'dem_elevation'
]]

# Preview
topo_features_clean.head()


,topo_features_id,profile_id,slope_code,altitude,aspect,land_surface_temp,dem_elevation
0,1,1/57,NaN,32.0,<NA>,<NA>,<NA>
1,2,1/59,D5,NaN,<NA>,<NA>,<NA>
2,3,1/61,NaN,NaN,<NA>,<NA>,<NA>
3,4,1/63,D1,1210.0,<NA>,<NA>,<NA>
4,5,1/64,NaN,NaN,<NA>,<NA>,<NA>


# Geological features table

In [52]:
# Load cleaned geo features data
geo_features_cleaning = pd.read_excel("/Users/inesschwartz/Desktop/Thesis/tables_soil_database/Perfis_local.xlsx")

# Rename columns
geo_features_cleaning.rename(columns={
    'PERFIL': 'profile_id',
    'GEOLOGIA': 'geology_id',
    'LITOLOGIA': 'lithology_id',
    'LITOLOGIA_1954': 'lithology_1954_id',
}, inplace=True)

# Add primary key column
geo_features_cleaning.insert(0, 'geo_features_id', range(1, len(geo_features_cleaning) + 1))

# make sure profile_id column is a string/varchar
geo_features_cleaning['profile_id'] = geo_features_cleaning['profile_id'].astype(str).str.strip().str[:20]


# Final normalized geo_features table (with codes as foreign keys)
geo_features_clean = geo_features_cleaning[[
    'geo_features_id',
    'profile_id',
    'geology_id',
    'lithology_id',
    'lithology_1954_id'
]]

#drop accents
import unicodedata
def remove_accents(text):
    if isinstance(text, str):
        # Normalize and remove diacritics
        text = unicodedata.normalize('NFKD', text)
        text = ''.join(c for c in text if not unicodedata.combining(c))
        return text
    return text

# Apply to all cells in the DataFrame
geo_features_clean = geo_features_clean.applymap(remove_accents)

#preview
geo_features_clean.head()

,geo_features_id,profile_id,geology_id,lithology_id,lithology_1954_id
0,1,1/57,NaN,d,NaN
1,2,1/59,Oendolongo,pp,Sistema do Maiombe
2,3,1/61,NaN,NaN,NaN
3,4,1/63,Karroo,Cs/Cal,Serie de Cassanje - T2'T1
4,5,1/64,NaN,NaN,NaN


In [24]:
## mapping tables
# Mappings for geology
geology_mapping = {
    "Kalahari": "Sistema do Kalahari",
    "Superficiais": "Formações Superficiais",
    "Karroo": "Sistema do Karroo",
    "Bembe": "Sistema do Bembe",
    "Oendolongo": "Sistema do Oendolongo",
    "Base": "Complexo de base",
    "Proterozóico": "Proterozóico",
    "Pleistocénico": "Pleistocénico",
    "Terciário": "Terciário (médio e inferior)",
    "TQ": "Quaternário e Terciário superior",
    "Cretácio": "",  # no description provided
    "RPKS": "Recente Plistocénico e Kalahari Superior"
}

# Mappings for lithology_1954
lithology_1954_mapping = {
    "γ": "Granitos, Granodioritos e Quartzodioritos",
    "PL": "Xistos, metaquartzitos, conglomerados, arcoses, ect.",
    "λ": "Rochas eruptivas indeterminadas",
    "δp": "Doleritos, doleritos pigeoníticos",
    "δab": "Diabases, diabases albito-cloriticas",
    "ε": "Noritos, gabros e peridotitos",
    "JK": "Composto de conglomerados, areias, cascalhos do Kalahari",
    "C": "Série Xisto - calcária",
    "Cal": "Sedimentos arenosos não consolidados",
    "K": "Série xisto - gresosa",
    "RT": "Não diferenciado",
    "σ": "Sienitos, sienitos nefelínicos",
    "Q": "Depósitos fossilíferos",
    "CS": "Grande conglomerado e série de Mwashya"
}

# Mappings for lithology
lithology_mapping = {
    "a": "Rochas arenáceas consolidadas",
    "aq": "Grés quartzíticos do Oendolongo",
    "b": "Rochas eruptivas básicas",
    "c": "Rochas sedimentares consolidadas calcárias",
    "c'": "Rochas sedimentares não consolidadas calcárias",
    "cg": "Rochas cristalofílicas argiláceas",
    "d": "Sedimentos não consolidados de origem marinha",
    "dc": "Depósitos coluvionares",
    "dr": "Diorítos",
    "e": "Rochas sedimentares consolidadas não calcárias",
    "g": "Rochas argiláceas consolidadas não calcárias",
    "g'": "Rochas argiláceas não consolidadas não calcárias",
    "g''": "Rochas cristalinas pouco micas em quartzo",
    "gp": "Rochas do  complexo gabro-plagioclastíco",
    "k": "Sedimentos não consolidados grosseiros do Kalahari",
    "m": "Rochas sedimentares não consolidadas calco-gipsíferas",
    "m'": "Depósitos coluvionares margosos",
    "mm": "Materiais mistos",
    "n": "Sedimentos não consolidados de origem continental",
    "nd": "não descrito",
    "pp": "Sedimentos não consolidados grosseiros plio-plistocénicos",
    "q": "Rochas cristalinas quartzíferas",
    "q'": "Materiais redistribuídos provenientes de desagregação rochas crist. quartzíferas",
    "qf": "Quartzitos ferruginosos do Oendolongo",
    "r": "Sedimentos grosseiros não especificados",
    "s": "Sienitos",
    "sx": "Formações (ou rochas) sedimentares não especificadas",
    "sx1": "Rochas sedimentares consolidadas com e sem calcário",
    "sx2": "Rochas sedimentares consolidadas",
    "v": "Materiais vulcânicos",
    "v'": "Rochas do complexo alcalino e/ou carboatítico",
    "x": "Rochas consolidadas não especificadas",
    "xm": "Xistos metamórficos",
    "xq": "Rochas cristalinas não especificadas",
    "z": "Rochas metassedimentares"
}

# Save each mapping as a DataFrame
pd.DataFrame([
    {"geology_code": k, "geology_description": v}
    for k, v in geology_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/geology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_code": k, "lithology_description": v}
    for k, v in lithology_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/lithology_mapping.csv", index=False)

pd.DataFrame([
    {"lithology_1954_code": k, "lithology_1954_description": v}
    for k, v in lithology_1954_mapping.items()
]).to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/lithology1954_mapping.csv", index=False)


# District table

In [54]:
import pandas as pd
import unicodedata

# Sample dataframe to illustrate
# samples = pd.read_csv(...)  # Uncomment and load your data if needed

# Final normalized districts table (with codes as foreign keys)
district_cleaning = samples.copy()

# Add primary key column
district_cleaning.insert(0, 'district_id', range(1, len(district_cleaning) + 1))

# Select relevant columns
district_clean = district_cleaning[[
    'district_id',
    'sample_id',
    'profile_id',
    'district'
]].copy()

# Strip whitespace from sample_id
district_clean.loc[:, 'sample_id'] = district_clean['sample_id'].astype(str).str.strip()

# make sure profile_id column is a string/varchar
district_clean['profile_id'] = district_clean['profile_id'].astype(str).str.strip().str[:20]

# Function to remove accents
def remove_accents(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text)
        return ''.join(c for c in text if not unicodedata.combining(c))
    return text

# Apply to all object (string) columns only
for col in district_clean.select_dtypes(include='object').columns:
    district_clean[col] = district_clean[col].apply(remove_accents)

# Preview
district_clean.head()


,district_id,sample_id,profile_id,district
0,1,630,139,Huambo
1,2,631,139,Huambo
2,3,632,139,Huambo
3,4,633,139,Huambo
4,5,687,208,Huambo


# Minerology info table

# Biology Info table

# Checking CSV Datatypes before DB export

In [55]:
import pandas as pd
import os

# Set the path to your folder containing the 10 CSV files
csv_folder = "/Users/inesschwartz/Desktop/Thesis/tables_clean"  

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Loop through each file and display column data types
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    print(f"\n📄 File: {file}")
    
    try:
        df = pd.read_csv(file_path)
        print(df.dtypes)
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")


📄 File: district_clean.csv
district_id     int64
sample_id       int64
profile_id     object
district       object
dtype: object

📄 File: soil_type_clean.csv
soil_type_id     int64
profile_id      object
grouping        object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object

📄 File: analyses_clean.csv
lab_sample_id      int64
sample_id        float64
EG               float64
thick_clay       float64
fine_clay        float64
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 68, dtype: object

📄 File: climate_features_clean.csv
climate_id                int64
profile_id               object
mean_annual_temp        float64
mean_annual_precip      float64
koppen_climate           object
thornthwaite_climate     object
hydric_regime            object
thermal_regime           object
dtype: object

📄 File: analyses_clean_check2.csv
lab_sample_id      

# Foreign key imports and datatype consistency

In [56]:
## standardize datatypes for identifiers
def convert_identifiers_to_string(df, id_columns):
    """
    Converts specified identifier columns in a DataFrame to string type,
    safely handling float64 values and preserving missing values (NA).
    Avoids SettingWithCopyWarning.
    """
    df = df.copy()  # ensure we're working with a copy, not a slice
    for col in id_columns:
        if col in df.columns:
            df.loc[:, col] = df[col].apply(
                lambda x: str(int(x)) if pd.notna(x) and isinstance(x, float) and x.is_integer()
                else str(x) if pd.notna(x)
                else pd.NA
            ).astype("string")
    return df

#usage

# Define identifier columns
identifier_columns = [
    'sample_id', 'site_info_id', 'profile_id',
    'horizon_id', 'lab_sample_id'
]

# Apply to each relevant dataframe
samples_check = convert_identifiers_to_string(samples_check, identifier_columns)
morphology_clean = convert_identifiers_to_string(morphology_clean, identifier_columns)
profile_clean = convert_identifiers_to_string(profile_clean, identifier_columns)
merged1 = convert_identifiers_to_string(merged1, identifier_columns)
site_info_clean = convert_identifiers_to_string(site_info_clean, identifier_columns)
district_clean = convert_identifiers_to_string(district_clean, identifier_columns)
soil_type_clean = convert_identifiers_to_string(soil_type_clean, identifier_columns)
geo_features_clean = convert_identifiers_to_string(geo_features_clean, identifier_columns)
topo_features_clean = convert_identifiers_to_string(topo_features_clean, identifier_columns)


In [57]:
# Check types of identifier columns in each dataframe
dfs = {
    "samples_check": samples_check,
    "morphology_clean": morphology_clean,
    "profile_clean": profile_clean,
    "merged1": merged1,
    "site_info_clean": site_info_clean,
    "district_clean": district_clean,
    "soil_type_clean": soil_type_clean,
    "geo_features_clean": geo_features_clean,
    "topo_features_clean": topo_features_clean,
}

for name, df in dfs.items():
    print(f"\n{name} column types:")
    for col in identifier_columns:
        if col in df.columns:
            print(f"  {col}: {df[col].dtype}")



samples_check column types:
  sample_id: object
  site_info_id: object
  profile_id: object
  horizon_id: object

morphology_clean column types:
  sample_id: object
  profile_id: object
  horizon_id: object

profile_clean column types:
  sample_id: object
  site_info_id: object
  profile_id: object

merged1 column types:
  sample_id: object
  lab_sample_id: object

site_info_clean column types:
  site_info_id: string
  profile_id: object

district_clean column types:
  sample_id: object
  profile_id: object

soil_type_clean column types:
  profile_id: object

geo_features_clean column types:
  profile_id: object

topo_features_clean column types:
  profile_id: object


In [58]:
## validate FK references in tables

# Check for missing FK references in morphology
missing_samples = samples_check[~samples_check['sample_id'].isin(morphology_clean['sample_id'])]

print(f"{len(missing_samples)} sample_id(s) in samples_clean are missing in morphology_horizon_clean.")
print(missing_samples[['sample_id']])


9636 sample_id(s) in samples_clean are missing in morphology_horizon_clean.
      sample_id
3           633
10          700
11          707
13          710
16          719
...         ...
14710     18867
14711     18868
14712     18869
14713     18870
14714     18871

[9636 rows x 1 columns]


In [59]:
# Samples table horizon_id FK 

# Drop old horizon_id if it exists (to avoid confusion)
samples_check = samples_check.drop(columns=['horizon_id'], errors='ignore')

# Merge horizon_id from morph into samples
samples_check = samples_check.merge(
    morphology_clean[['sample_id', 'horizon_id']],
    on='sample_id',
    how='left'
)

missing = samples_check[samples_check['horizon_id'].isna()]
print(f"{len(missing)} samples could not be matched with a horizon_id.")


9636 samples could not be matched with a horizon_id.


In [60]:

# Re-order and select relevant columns
samples_clean = samples_check[[
    'sample_id',
    'site_info_id',  
    'profile_id',
    'horizon_id',
    'year',
    'shelf',
    'Room'  # Ensure this matches the column name in your DataFrame
]].copy()

# Preview the result
samples_clean.head()


,sample_id,site_info_id,profile_id,horizon_id,year,shelf,Room
0,630,172,139,Hb_139/46_1_1,1946.0,1,22
1,631,173,139,Hb_139/46_2_1,1946.0,1,22
2,632,174,139,Hb_139/46_3_1,1946.0,1,22
3,633,175,139,NaN,1946.0,1,22
4,687,1034,208,Hb_208/46_1_1,1946.0,1,22


In [61]:
# Drop old FK columns if they exist
profile_clean = profile_clean.drop(columns=['site_info_id', 'sample_id', 'soil_type_id'], errors='ignore')

# Merge site_info_id from site_info_clean
profile_clean = profile_clean.merge(
    site_info_clean[['profile_id', 'site_info_id']],
    on='profile_id',
    how='left'
)

# Merge sample_id from samples_check
profile_clean = profile_clean.merge(
    samples_check[['profile_id', 'sample_id']],
    on='profile_id',
    how='left'
)

# Merge soil_type_id from soil_type_clean
profile_clean = profile_clean.merge(
    soil_type_clean[['profile_id', 'soil_type_id']],
    on='profile_id',
    how='left'
)

# Make unique record ID for profile info
profile_clean.insert(0, 'profile_record_id', range(1, len(profile_clean) + 1))

# Reorder columns to match SAMPLES schema
column_order = [
    'profile_record_id',
    'profile_id',
    'site_info_id',
    'sample_id',
    'soil_type_id'
]
profile_clean = profile_clean.loc[:, column_order]

# Check for unmatched profile_ids in each join (optional)
missing_site = profile_clean[profile_clean['site_info_id'].isna()]
missing_sample = profile_clean[profile_clean['sample_id'].isna()]
missing_soil = profile_clean[profile_clean['soil_type_id'].isna()]

print(f"{len(missing_site)} profiles missing site_info_id")
print(f"{len(missing_sample)} profiles missing sample_id")
print(f"{len(missing_soil)} profiles missing soil_type_id")

profile_clean.head()

155 profiles missing site_info_id
1288 profiles missing sample_id
0 profiles missing soil_type_id


,profile_record_id,profile_id,site_info_id,sample_id,soil_type_id
0,1,1/51,<NA>,NaN,1
1,2,1/57,1,4835,2
2,3,1/57,1,4836,2
3,4,1/57,1,4837,2
4,5,1/57,1,4838,2


In [62]:
# Drop old FKs if they exist (only needs to be done once)
site_info_clean = site_info_clean.drop(
    columns=['land_cover_id', 'climate_id', 'geology_id', 'topo_feature_id', 'district_id', 'geo_features_id', 'topo_features_id'],
    errors='ignore'
)

# Merge climate_id from climate_features_clean
site_info_clean = site_info_clean.merge(
    climate_features_clean[['profile_id', 'climate_id']],
    on='profile_id',
    how='left'
)

# Merge topo_features_id from topo_features_clean
site_info_clean = site_info_clean.merge(
    topo_features_clean[['profile_id', 'topo_features_id']],
    on='profile_id',
    how='left'
)

# Merge district_id from district_clean
site_info_clean = site_info_clean.merge(
    district_clean[['profile_id', 'district_id']],
    on='profile_id',
    how='left'
)

# Merge geo_features_id from geo_features_clean
site_info_clean = site_info_clean.merge(
    geo_features_clean[['profile_id', 'geo_features_id']],
    on='profile_id',
    how='left'
)


# Optional: Check for missing FKs
missing_climate = site_info_clean[site_info_clean['climate_id'].isna()]
missing_topo = site_info_clean[site_info_clean['topo_features_id'].isna()]
missing_district = site_info_clean[site_info_clean['district_id'].isna()]
missing_geology = site_info_clean[site_info_clean['geo_features_id'].isna()]

print(f"{len(missing_climate)} records missing climate_id")
print(f"{len(missing_topo)} records missing topo_features_id")
print(f"{len(missing_district)} records missing district_id")
print(f"{len(missing_geology)} records missing geo_features_id")


0 records missing climate_id
0 records missing topo_features_id
2178 records missing district_id
0 records missing geo_features_id


# SAVING CLEANED TABLES TO CSV FOR DB

In [63]:
samples_check.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/samples_clean.csv", index=False)
merged1.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/analyses_clean.csv", index=False)
profile_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/profile_clean.csv", index=False)
morphology_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/morphology_horizon_clean.csv", index=False)
soil_type_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/soil_type_clean.csv", index=False)
site_info_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/site_info_clean.csv", index=False)
climate_features_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/climate_features_clean.csv", index=False)
topo_features_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/topo_features_clean.csv", index=False)
geo_features_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/geo_features_clean.csv", index=False)
district_clean.to_csv("/Users/inesschwartz/Desktop/Thesis/tables_clean/district_clean.csv", index=False)

In [64]:
# Save random 100 rows of each table as mini versions for DB test
import os

# Create folder if it doesn't exist
mini_path = "/Users/inesschwartz/Desktop/Thesis/tables_clean_mini"
os.makedirs(mini_path, exist_ok=True)

# Save random 100 rows of each table as mini versions
samples_check.sample(n=100, random_state=42).to_csv(f"{mini_path}/samples.csv", index=False)
merged1.sample(n=100, random_state=42).to_csv(f"{mini_path}/analyses.csv", index=False)
profile_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/profile.csv", index=False)
morphology_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/morphology_horizon.csv", index=False)
soil_type_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/soil_type.csv", index=False)
site_info_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/site_info.csv", index=False)
climate_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/climate_feat.csv", index=False)
topo_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/topo_feat.csv", index=False)
geo_features_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/geo_feat.csv", index=False)
district_clean.sample(n=100, random_state=42).to_csv(f"{mini_path}/districts.csv", index=False)



In [65]:
tables = {
    "samples": samples_check,
    "analyses": merged1,
    "profile": profile_clean,
    "morphology_horizon": morphology_clean,
    "soil_type": soil_type_clean,
    "site_info": site_info_clean,
    "climate_feat": climate_features_clean,
    "topo_feat": topo_features_clean,
    "geo_feat": geo_features_clean,
    "districts": district_clean
}

for name, df in tables.items():
    df.replace("", pd.NA, inplace=True)
    df.to_csv(f"/Users/inesschwartz/Desktop/Thesis/tables_clean_mini/{name}.csv", index=False, na_rep="NULL")


/var/folders/tp/79mdnyy56_xc3g1jvp9wf4_80000gn/T/ipykernel_1816/2163122740.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace("", pd.NA, inplace=True)


In [37]:
# Step 1: Count duplicates
duplicates = profile_clean[profile_clean.duplicated('profile_record_id', keep=False)]

# Step 2: Sort for clarity (optional)
duplicates_sorted = duplicates.sort_values('profile_record_id')

# Step 3: Print or inspect
print("🔁 Duplicate `profile_record_id` values and their rows:")
print(duplicates_sorted)


🔁 Duplicate `profile_record_id` values and their rows:
Empty DataFrame
Columns: [profile_record_id, profile_id, site_info_id, sample_id, soil_type_id]
Index: []
